### **Package**

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
import re


def parse_vry_file(filename):
    coordinates = []
    demands = []
    capacity = None
    reading_coords = False
    reading_demands = False

    with open(filename, 'r') as file:
        for line in file:
            if line.strip() == "NODE_COORD_SECTION":
                reading_coords = True
                continue
            elif line.strip() == "DEMAND_SECTION":
                reading_coords = False
                reading_demands = True
                continue
            elif line.strip() == "DEPOT_SECTION":
                reading_demands = False
                continue
            elif "CAPACITY" in line:
                capacity = int(line.split()[-1])

            if reading_coords:
                parts = line.split()
                if len(parts) >= 3:
                    coordinates.append((float(parts[1]), float(parts[2])))
            elif reading_demands:
                parts = line.split()
                if len(parts) == 2:
                    demands.append(int(parts[1]))

    coordinates = np.array(coordinates)
    demands = np.array(demands)
    distance_matrix = cdist(coordinates, coordinates, metric='euclidean')

    return distance_matrix, demands, capacity

# if __name__ == "__main__":
#     filename = "/content/A-n53-k7.vrp"  # Replace with your actual file name
#     distance_matrix, demands, capacity, num_trucks = parse_vry_file(filename)
#     print("Distance Matrix:")
#     print(distance_matrix)
#     print("Demands:", demands)
#     print("Capacity:", capacity)
#     print("Number of Trucks:", num_trucks)


In [ ]:
def parse_vry(filename):
    n_truc=0

    with open(filename, 'r') as file:
        for line in file:
          n_truc+=1

    return n_truc-1

In [ ]:
# if __name__ == "__main__":
#     filename = "/content/vrplib/X-n200-k36.sol"  # Replace with your actual file name
#     num_trucks = parse_vry(filename)
#     print("Number of Trucks:", num_trucks)


In [ ]:
!pip install rl4co[graph] # include torch-geometric
# !pip install vrplib # for reading instance files
!pip install ortools matplotlib

# NOTE: to install latest version from Github (may be unstable) install from source instead:
!pip install git+https://github.com/ai4co/rl4co.git

  Cloning https://github.com/ai4co/rl4co.git to /tmp/pip-req-build-szgrpkja
  Running command git clone --filter=blob:none --quiet https://github.com/ai4co/rl4co.git /tmp/pip-req-build-szgrpkja
  Resolved https://github.com/ai4co/rl4co.git to commit 0e69192d26a9101c533992ef6f1070aad45d309f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Install the `vrplib` package
!pip install vrplib

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import torch
import vrplib
from tensordict import TensorDict

from rl4co.envs import CVRPEnv
from rl4co.models.zoo.am import AttentionModelPolicy
from rl4co.models.rl import REINFORCE
from rl4co.utils.trainer import RL4COTrainer

from tqdm import tqdm

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# RL4CO env based on TorchRL
env = CVRPEnv(generator_params={'num_loc': 50})

# Policy: neural network, in this case with encoder-decoder architecture
.3

policy = AttentionModelPolicy(env_name=env.name).to(device)

# RL Model: REINFORCE and greedy rollout baseline
model = REINFORCE(env,
                    policy,
                    baseline="rollout",
                    batch_size=512,
                    train_data_size=100_000,
                    val_data_size=10_000,
                    optimizer_kwargs={"lr": 1e-4},
                    )

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.


### Download vrp problems

In [ ]:
instances=[ 'Li_21','Li_22','Li_23','Li_24','Li_25','Li_26','Li_27','Li_28','Li_29','Li_30','Li_31','Li_32']


In [ ]:
# instances=['A-n53-k7',
#  'A-n54-k7',
#  'A-n55-k9',
#  'A-n60-k9',
#  'A-n61-k9',
#  'A-n62-k8',
#  'A-n63-k9',
#  'A-n63-k10',
#  'A-n64-k9',
#  'A-n65-k9',
#  'A-n69-k9',
#  'A-n80-k10',
#  'B-n51-k7',
#  'B-n52-k7',
#  'B-n56-k7',
#  'B-n57-k9',
#  'B-n63-k10',
#  'B-n64-k9',
#  'B-n66-k9',
#  'B-n67-k10',
#  'B-n68-k9',
#  'B-n78-k10',
#  'E-n51-k5',
#  'E-n76-k7',
#  'E-n76-k8',
#  'E-n76-k10',
#  'E-n76-k14',
#  'E-n101-k8',
#  'E-n101-k14',
#  'F-n72-k4',
#  'M-n101-k10']

In [ ]:
# problem_names = vrplib.list_names(low=50, high=100, vrp_type='cvrp')

# instances = [] # Collect Set A, B, E, F, M datasets
# for name in problem_names:
#     if 'A' in name:
#         instances.append(name)
#     elif 'B' in name:
#         instances.append(name)
#     elif 'E' in name:
#         instances.append(name)
#     elif 'F' in name:
#         instances.append(name)
#     elif 'M' in name and 'CMT' not in name:
#         instances.append(name)


# instances=['M-n200-k16',
#  'M-n200-k17',
#  'Golden_13',
#  'Golden_14',
#  'Golden_17',
#  'Golden_18',
#  'Golden_19',
#  'Golden_20',
#  'X-n181-k23',
#  'X-n186-k15',
#  'X-n190-k8',
#  'X-n195-k51']
# instances= ['X-n204-k19',
#  'X-n209-k16',
#  'X-n214-k11',
#  'X-n219-k73',
#  'X-n223-k34',
#  'X-n228-k23',
#  'X-n233-k16',
#  'X-n237-k14',
#  'X-n242-k48',
#  'X-n247-k50',
#  'X-n251-k28',
#  'X-n256-k16',
#  'X-n261-k13',
#  'X-n266-k58',
#  'X-n270-k35',
#  'X-n275-k28',
#  'X-n284-k15',
#  'X-n289-k60',
#  'X-n294-k50',
#  'X-n298-k31',
#  'X-n317-k53',
#  'X-n322-k28',
#  'X-n331-k15',
#  'X-n344-k43',
#  'X-n351-k40',
#  'X-n367-k17',
#  'X-n376-k94',
#  'X-n393-k38',
#  'X-n420-k130',
#  'X-n439-k37',
#  'X-n469-k138']
# # Modify the path you want to save
# # Note: we don't have to create this folder in advance
# instances=['Li_30','Li_31','Li_32']
path_to_save = './vrplib/'

try:
    os.makedirs(path_to_save)
    for instance in tqdm(instances):
        vrplib.download_instance(instance, path_to_save)
        vrplib.download_solution(instance, path_to_save)
except: # already exist
    pass

In [ ]:
len(instances)

12

In [ ]:
# Utils function: we will normalize the coordinates of the VRP instances
def normalize_coord(coord:torch.Tensor) -> torch.Tensor:
    x, y = coord[:, 0], coord[:, 1]
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()

    x_scaled = (x - x_min) / (x_max - x_min)
    y_scaled = (y - y_min) / (y_max - y_min)
    coord_scaled = torch.stack([x_scaled, y_scaled], dim=1)
    return coord_scaled

def vrplib_to_td(problem, normalize=True):
    coords = torch.tensor(problem['node_coord']).float()
    coords_norm = normalize_coord(coords) if normalize else coords
    demand = torch.tensor(problem['demand'][1:]).float()
    capacity = problem['capacity']
    n = coords.shape[0]
    td = TensorDict({
        'depot': coords_norm[0,:],
        'locs': coords_norm[1:,:],
        'demand': demand / capacity, # normalized demand
        'capacity': capacity, # original capacity, not needed for inference
    })
    td = td[None] # add batch dimension, in this case just 1
    return td

### Train

We will train for few steps just to show the effects of training a model.
Alternatively, we can load the a pretrained checkpoint, e.g. with:

```python
model = AttentionModel.load_from_checkpoint(checkpoint_path, load_baseline=False)
```


In [ ]:
import time

In [ ]:
trainer = RL4COTrainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=None,
)
start=time.time()
trainer.fit(model)
stop=time.time()-start

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name     | Type                 | Params | Mode 
----------------------------------------------------------
0 | env      | CVRPEnv              | 0      | train
1 | policy   | AttentionModelPolicy | 694 K  | train
2 | baseline | WarmupBaseline       | 694 K  | train
----------------------------------------------------------
1.4 M     Traina

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
print(stop)

272.7405834197998


In [ ]:
torch.save(model.state_dict(), "cvrp_rl4co_trained.pth")
print("✅ Model Trained & Saved!")

✅ Model Trained & Saved!


In [ ]:
import os
import torch
import vrplib
import numpy as np
from tqdm import tqdm
from rl4co.envs import CVRPEnv
from rl4co.models.zoo.am import AttentionModelPolicy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Test trained model

In [ ]:
import os
import torch
import vrplib
import numpy as np
from tqdm import tqdm
from rl4co.envs import CVRPEnv
from rl4co.models.zoo.am import AttentionModelPolicy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # RL4CO env
# env = CVRPEnv(generator_params={'num_loc': 50})
# policy = AttentionModelPolicy(env_name=env.name).to(device).eval()

# Define number of solutions to generate
num_samples_list = [64, 128, 256, 512, 1024]
policy = model.policy.to(device).eval() # trained policy
bkss=[]
gap_64=[]
gap_128=[]
gap_256=[]
gap_512=[]
gap_1024=[]

# Single problem instance
for instance in instances:
# instance = 'A-n53-k7'  # Replace with actual instance name
  problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))
  optimal_cost = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))['cost']
  bkss.append(optimal_cost)

  # gap_results = {}
  print(instance)
  for num_samples in num_samples_list:
      td_reset = env.reset(vrplib_to_td(problem).to(device))

      with torch.inference_mode():
          out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=num_samples, select_best=True)
          unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
          cost = -env.get_reward(unnormalized_td, out["actions"]).cpu().numpy()

      gaps = (cost - optimal_cost) / optimal_cost
      # print(gaps)
      median_gap = np.median(gaps)
      if num_samples==64:
        gap_64.append(median_gap)
      if num_samples==128:
        gap_128.append(median_gap)
      if num_samples==256:
        gap_256.append(median_gap)
      if num_samples==512:
        gap_512.append(median_gap)
      if num_samples==1024:
        gap_1024.append(median_gap)
      # gap_results[num_samples] = median_gap
      # print(f"Num Samples: {num_samples:<5} Median Gap: {median_gap:.2%}")

  # # Print results
  # print("\nSummary of Median Gaps:")
  # for num_samples, median_gap in gap_results.items():
  #     print(f"{num_samples} samples -> Median Gap: {median_gap:.2%}")


In [ ]:
import pandas as pd
df = pd.DataFrame({
    'Instance': instances,
    # 'No: of nodes': no_nodes,
    'BKS': bkss,
    'gap_64': gap_64,
    'gap_128': gap_128,
    'gap_256': gap_256,
    'gap_512': gap_512,
    'gap_1024': gap_1024,

})

In [ ]:
df.to_csv('Result2.csv', index=False)

In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def create_data_model(d,demand,cap,n_trucks):
    """Stores the data for the CVRP problem."""
    data = {
        'distance_matrix': d,
        'demands': demand,  # Demand at each node (0 is the depot)
        'vehicle_capacities': [cap for _ in range(n_trucks)],  # Capacity of each vehicle
        'num_vehicles': n_trucks,
        'depot': 0  # Starting point for all vehicles
    }
    return data

def print_solution(data, manager, routing, solution):
    """Prints the solution to the console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        # route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            # route_load += data['demands'][node_index]
            plan_output += f' {node_index} ->'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        # plan_output += ' 0\n'
        # plan_output += f'Load of the route: {route_load}\n'
        # plan_output += f'Distance of the route: {route_distance}m\n'
        # print(plan_output)
        total_distance += route_distance
        # total_load += route_load
    # print(f'Total distance of all routes: {total_distance}m')
    return total_distance
    # print(f'Total load of all routes: {total_load}')

# if __name__ == '__main__':
#     main()


In [ ]:
policy = model.policy.to(device).eval() # trained policy

tds, actions = [], []
t_rl=[]
t_or=[]
ans_rl=[]
ans_or=[]
no_nodes=[]
bks=[]
gap_rl=[]
gap_or=[]
for instance in instances:
    # Inference
    problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))
    start_rl=time.time()
    td_reset = env.reset(vrplib_to_td(problem).to(device))
    with torch.inference_mode():
        out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=128, select_best=True)
        unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
        cost = -env.get_reward(unnormalized_td, out["actions"]).int().item() # unnormalized cost
    stop_rl=time.time()-start_rl
    t_rl.append(stop_rl)

    # Google_OR_tools

    def convert_to_int(matrix):
      return [[int(element) for element in row] for row in matrix]


    filename = os.path.join(path_to_save, instance+'.vrp')  # Replace with your actual file name
    d,demand,cap = parse_vry_file(filename)
    filenam = os.path.join(path_to_save, instance+'.sol')
    n_truck=parse_vry(filenam)
    d = convert_to_int(d)
    no_nodes.append(len(demand))
    start_or = time.time()
    data = create_data_model(d,demand,cap,n_truck)

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                            data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    def distance_callback(from_index, to_index):
        # Returns the distance between the two nodes
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint
    def demand_callback(from_index):
        # Returns the demand of the node
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    stop_or=time.time()-start_or
    t_or.append(stop_or)

    # Print solution on console
    if solution:
        ans=print_solution(data, manager, routing, solution)
    else:
        ans= 1e10

    # Load the optimal cost
    sol = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
    optimal_cost = sol['cost']
    bks.append(optimal_cost)
    ans_rl.append(cost)
    ans_or.append(ans)


    tds.append(td_reset)
    actions.append(out["actions"])
    # Calculate the gap and print
    rl_gap = (cost - optimal_cost) / optimal_cost
    or_gap = (ans - optimal_cost) / optimal_cost

    gap_rl.append(rl_gap)
    gap_or.append(or_gap)
    print(f'Problem: {instance:<15} Cost_rl: {cost:<8} cost_or:{ans:<8} BKS: {optimal_cost:<8}\t Gap_rl: {rl_gap:.2%} Gap_or: {or_gap:.2%}')

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'Instance': instances,
    'No: of nodes': no_nodes,
    'BKS': bks,
    'Google OR sol': ans_or,
    'RL sol': ans_rl,
    '% Gap Google OR': gap_or,
    '% Gap RL': gap_rl,
    'Time Google OR': t_or,
    'Time RL': t_rl
})

In [ ]:
df.to_csv('Result1.csv', index=False)

In [ ]:
def separate_vehicle_routes(route_sequence):
    """
    Separate a single sequence into individual vehicle routes

    :param route_sequence: List of nodes including depot visits
    :return: List of routes for each vehicle
    """
    vehicle_routes = []
    current_route = []

    # Remove trailing zeros at the end
    while route_sequence and route_sequence[-1] == 0:
        route_sequence.pop()

    for node in route_sequence:
        if node == 0:
            if current_route:  # If we have nodes in the current route
                vehicle_routes.append([0] + current_route + [0])  # Add depot at start and end
                current_route = []
        else:
            current_route.append(node)

    # Add the last route if there are remaining nodes
    if current_route:
        vehicle_routes.append([0] + current_route + [0])

    return vehicle_routes

def print_and_save_routes(instance_name, route_sequence, cost, optimal_cost, output_file="routes.txt"):
    """
    Print and save properly separated vehicle routes

    :param instance_name: Name of the problem instance
    :param route_sequence: Original sequence of nodes
    :param cost: Solution cost
    :param optimal_cost: Best known solution cost
    :param output_file: File to save the routes
    """
    # Convert route sequence to list if it's not already
    if isinstance(route_sequence, str):
        route_sequence = [int(x.strip()) for x in route_sequence.split('->')]

    # Separate into vehicle routes
    vehicle_routes = separate_vehicle_routes(route_sequence)

    # Print to console
    print(f"\nProblem: {instance_name}")
    print(f"Cost: {cost}")
    print(f"BKS: {optimal_cost}")
    print(f"Gap: {((cost - optimal_cost) / optimal_cost):.2%}")
    print("Routes:")
    for i, route in enumerate(vehicle_routes, 1):
        route_str = ' -> '.join(str(node) for node in route)
        print(f"Vehicle {i}: {route_str}")

    # Save to file
    with open(output_file, 'a') as f:
        f.write(f"\nProblem: {instance_name}\n")
        f.write(f"Cost: {cost}\n")
        f.write(f"BKS: {optimal_cost}\n")
        f.write(f"Gap: {((cost - optimal_cost) / optimal_cost):.2%}\n")
        f.write("Routes:\n")
        for i, route in enumerate(vehicle_routes, 1):
            route_str = ' -> '.join(str(node) for node in route)
            f.write(f"Vehicle {i}: {route_str}\n")
        f.write("-" * 50 + "\n")

# Example usage with your data
route = "39 -> 3 -> 5 -> 14 -> 13 -> 52 -> 24 -> 25 -> 0 -> 1 -> 33 -> 6 -> 20 -> 31 -> 27 -> 51 -> 0 -> 4 -> 47 -> 21 -> 34 -> 11 -> 41 -> 9 -> 0 -> 28 -> 22 -> 46 -> 35 -> 38 -> 18 -> 0 -> 17 -> 7 -> 12 -> 16 -> 32 -> 15 -> 19 -> 48 -> 23 -> 50 -> 36 -> 0 -> 8 -> 40 -> 26 -> 10 -> 29 -> 49 -> 44 -> 30 -> 0 -> 45 -> 42 -> 43 -> 2 -> 37 -> 0 -> 0 -> 0 -> 0"

print_and_save_routes("A-n53-k7", route, 1158, 1010)

In [ ]:
import os
import torch
import vrplib

policy = model.policy.to(device).eval()  # trained policy

tds, actions = [], []
results = []  # Store results for writing to file

for instance in instances:
    # Inference
    problem = vrplib.read_instance(os.path.join(path_to_save, instance + '.vrp'))
    td_reset = env.reset(vrplib_to_td(problem).to(device))

    with torch.inference_mode():
        out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=128, select_best=True)
        unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
        cost = -env.get_reward(unnormalized_td, out["actions"]).int().item()  # unnormalized cost

    # Load the optimal cost
    solution = vrplib.read_solution(os.path.join(path_to_save, instance + '.sol'))
    optimal_cost = solution['cost']

    tds.append(td_reset)
    actions.append(out["actions"])

    # Extract routes
    routes = out["actions"].cpu().numpy().tolist()  # Convert to list for easier processing

    # Store result
    result_str = f'Problem: {instance} Cost: {cost} BKS: {optimal_cost} Gap: {(cost - optimal_cost) / optimal_cost:.2%}\n'
    result_str += "Routes:\n"

    for idx, route in enumerate(routes):
        route_str = " -> ".join(map(str, route))
        result_str += f'Vehicle {idx + 1}: {route_str}\n'

    # results.append(result_str)
    print_and_save_routes(instance, route_str, cost, optimal_cost)

    # # Print results
    # print(result_str)

# # Save to text file
# output_file = os.path.join(path_to_save, "routes.txt")
# with open(output_file, "w") as f:
#     f.write("\n".join(results))

# print(f"Routes saved to {output_file}")


In [ ]:
# Plot some instances
env.render(tds[0], actions[0].cpu())
env.render(tds[-2], actions[-2].cpu())
env.render(tds[-1], actions[-1].cpu())

Great! We can see that the performance vastly improved even with just few minutes of training.

There are several ways to improve the model's performance further, such as:
- Training for more steps
- Using a different model architecture
- Using a different training algorithm
- Using a different hyperparameters
- Using a different `Generator`
- ... and many more!

In [ ]:
from sklearn.manifold import MDS
import numpy as np

# Example: Replace this with your actual cost matrix (N x N)
cost_matrix = np.array([[ 0, 73, 23, 98, 51],
        [73,  0, 64, 34,  8],
        [23, 64,  0,  9, 94],
        [98, 34,  9,  0, 75],
        [51,  8, 94, 75,  0]])

# Convert cost matrix into pseudo-coordinates (2D)
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)
pseudo_coords = mds.fit_transform(cost_matrix)

print(pseudo_coords)  # These can be used as node coordinates

### Varying the training size

In [ ]:
import pandas as pd

In [ ]:
import time

In [ ]:
arr=[50,100,200,500]
times=[]
dum = {'Problem': instances}
data_t = pd.DataFrame(dum)
data_a = pd.DataFrame(dum)

for n in arr:
  print(n)
  env = CVRPEnv(generator_params={'num_loc': n})

# Policy: neural network, in this case with encoder-decoder architecture
  policy = AttentionModelPolicy(env_name=env.name).to(device)

  # RL Model: REINFORCE and greedy rollout baseline
  model = REINFORCE(env,
                      policy,
                      baseline="rollout",
                      batch_size=512,
                      train_data_size=100_000,
                      val_data_size=10_000,
                      optimizer_kwargs={"lr": 1e-4},
                      )
  trainer = RL4COTrainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=None,
  )
  start=time.time()
  trainer.fit(model)
  stop=time.time()-start
  times.append(stop)


  policy = model.policy.to(device).eval() # trained policy

  tds, actions = [], []
  t_rl=[]
  ans_rl=[]
  bks=[]
  gap_rl=[]
  for instance in instances:
      # Inference
      problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))
      start_rl=time.time()
      td_reset = env.reset(vrplib_to_td(problem).to(device))
      with torch.inference_mode():
          out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=128, select_best=True)
          unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
          cost = -env.get_reward(unnormalized_td, out["actions"]).int().item() # unnormalized cost
      stop_rl=time.time()-start_rl
      t_rl.append(stop_rl)

      # Load the optimal cost
      sol = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
      optimal_cost = sol['cost']
      bks.append(optimal_cost)
      ans_rl.append(cost)


      tds.append(td_reset)
      actions.append(out["actions"])
      # Calculate the gap and print
      rl_gap = (cost - optimal_cost) / optimal_cost
      gap_rl.append(rl_gap)
  t=f'Time-{n}'
  a=f'Sol-{n}'
  g=f'Gap={n}'
  data_t[t]=t_rl
  data_a[a]=ans_rl
  data_a[g]=gap_rl
      # print(f'Problem: {instance:<15} Cost_rl: {cost:<8} BKS: {optimal_cost:<8}\t Gap_rl: {rl_gap:.2%} Gap_or: {or_gap:.2%}')



In [ ]:
data_t.to_csv('Time.csv', index=False)

In [ ]:
data_a.to_csv('Sol.csv', index=False)

In [ ]:
times

### RL solution for Google-OR

In [ ]:
import os
import torch
import vrplib
import numpy as np
from tqdm import tqdm
from rl4co.envs import CVRPEnv
from rl4co.models.zoo.am import AttentionModelPolicy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy = model.policy.to(device).eval() # trained policy

In [ ]:
import os
import torch
import vrplib
import numpy as np
from tqdm import tqdm
from rl4co.envs import CVRPEnv
from rl4co.models.zoo.am import AttentionModelPolicy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy = model.policy.to(device).eval() # trained policy

tds, actions = [], []
t_rl=[]
t_or=[]
ans_rl=[]
ans_or=[]
no_nodes=[]
bks=[]
gap_rl=[]
gap_or=[]
for instance in instances:
    # Inference
    problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))
    start_rl=time.time()
    td_reset = env.reset(vrplib_to_td(problem).to(device))
    with torch.inference_mode():
        out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=128, select_best=True)
        unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
        cost = -env.get_reward(unnormalized_td, out["actions"]).int().item() # unnormalized cost
    stop_rl=time.time()-start_rl
    t_rl.append(stop_rl)

    # Google_OR_tools

    def convert_to_int(matrix):
      return [[int(element) for element in row] for row in matrix]


    filename = os.path.join(path_to_save, instance+'.vrp')  # Replace with your actual file name
    d,demand,cap = parse_vry_file(filename)
    filenam = os.path.join(path_to_save, instance+'.sol')
    n_truck=parse_vry(filenam)
    d = convert_to_int(d)
    no_nodes.append(len(demand))
    start_or = time.time()
    data = create_data_model(d,demand,cap,n_truck)

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                            data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    def distance_callback(from_index, to_index):
        # Returns the distance between the two nodes
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint
    def demand_callback(from_index):
        # Returns the demand of the node
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    stop_or=time.time()-start_or
    t_or.append(stop_or)

    # Print solution on console
    if solution:
        ans=print_solution(data, manager, routing, solution)
    else:
        ans= 1e10

    # Load the optimal cost
    sol = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
    optimal_cost = sol['cost']
    bks.append(optimal_cost)
    ans_rl.append(cost)
    ans_or.append(ans)


    tds.append(td_reset)
    actions.append(out["actions"])
    # Calculate the gap and print
    rl_gap = (cost - optimal_cost) / optimal_cost
    or_gap = (ans - optimal_cost) / optimal_cost

    gap_rl.append(rl_gap)
    gap_or.append(or_gap)
    print(f'Problem: {instance:<15} Cost_rl: {cost:<8} cost_or:{ans:<8} BKS: {optimal_cost:<8}\t Gap_rl: {rl_gap:.2%} Gap_or: {or_gap:.2%}')

Problem: Li_21           Cost_rl: 54215    cost_or:17419    BKS: 16212.82548	 Gap_rl: 234.40% Gap_or: 7.44%
Problem: Li_22           Cost_rl: 42815    cost_or:14687    BKS: 14499.04468	 Gap_rl: 195.30% Gap_or: 1.30%
Problem: Li_23           Cost_rl: 71630    cost_or:20201    BKS: 18801.12848	 Gap_rl: 280.99% Gap_or: 7.45%
Problem: Li_24           Cost_rl: 91208    cost_or:22920    BKS: 21389.43049	 Gap_rl: 326.42% Gap_or: 7.16%
Problem: Li_25           Cost_rl: 55138    cost_or:15637    BKS: 16665.7 	 Gap_rl: 230.85% Gap_or: -6.17%
Problem: Li_26           Cost_rl: 112793   cost_or:25657    BKS: 23977.73256	 Gap_rl: 370.41% Gap_or: 7.00%
Problem: Li_27           Cost_rl: 61410    cost_or:16391    BKS: 17320.0 	 Gap_rl: 254.56% Gap_or: -5.36%
Problem: Li_28           Cost_rl: 135665   cost_or:30404    BKS: 26566.03399	 Gap_rl: 410.67% Gap_or: 14.45%
Problem: Li_29           Cost_rl: 166672   cost_or:32239    BKS: 29154.33598	 Gap_rl: 471.69% Gap_or: 10.58%
Problem: Li_30           Cost_

In [ ]:
import pandas as pd
# df = pd.DataFrame({
#     'Instance': instances,
#     'No: of nodes': no_nodes,
#     'BKS': bks,
#     'Google OR sol': ans_or,
#     'RL sol': ans_rl,
#     '% Gap Google OR': gap_or,
#     '% Gap RL': gap_rl,
#     'Time Google OR': t_or,
#     'Time RL': t_rl
# })

In [ ]:
df.to_csv('Result_a.csv', index=False)

In [ ]:
from google.colab import files
files.download('Result_a.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
instance= instances[]

In [ ]:
# print(instance)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# policy = model.policy.to(device).eval()  # trained policy
# problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))
# # start_rl=time.time()
# td_reset = env.reset(vrplib_to_td(problem).to(device))
# with torch.inference_mode():
#     out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=128, select_best=True)
#     unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
#     cost = -env.get_reward(unnormalized_td, out["actions"]).int().item() # unnormalized cost
# # stop_rl=time.time()-start_rl
# # t_rl.append(stop_rl)

In [ ]:
# rl_solution = out["actions"][0].tolist()  # Convert tensor to list
# print("RL4CO Solution:", rl_solution)

In [ ]:
def convert_to_int(matrix):
  return [[int(element) for element in row] for row in matrix]

In [ ]:
# filename = os.path.join(path_to_save, instance+'.vrp')  # Replace with your actual file name
# d,demand,cap = parse_vry_file(filename)
# filenam = os.path.join(path_to_save, instance+'.sol')
# n_truck=parse_vry(filenam)
# d = convert_to_int(d)

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

def create_data_model(distance_matrix, demands, vehicle_capacity, depot=0):
    """Create the data dictionary for the CVRP problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['demands'] = demands
    data['vehicle_capacities'] = [vehicle_capacity] * get_num_vehicles(rl_solution)
    data['num_vehicles'] = len(data['vehicle_capacities'])
    data['depot'] = depot
    return data

def get_num_vehicles(solution):
    """Count number of routes in the RL solution (number of depot visits)."""
    return sum(1 for x in solution if x == 0) - 1  # Subtract 1 as last depot visit doesn't count

def convert_rl_solution_to_routes(rl_solution):
    """Convert the flat RL solution into separate routes."""
    routes = []
    current_route = []

    for node in rl_solution:
        if node == 0 and current_route:  # Depot
            routes.append(current_route)
            current_route = []
        elif node != 0:  # Customer
            current_route.append(node)

    if current_route:  # Add last route if exists
        routes.append(current_route)

    return routes

def create_initial_solution(manager, routing, rl_solution):
    """Creates an initial solution based on the RL solution."""
    routes = convert_rl_solution_to_routes(rl_solution)

    # Create the initial solution
    initial_solution = routing.ReadAssignmentFromRoutes(
        routes, True)

    return initial_solution

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        # plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            # node_index = manager.IndexToNode(index)
            # route_load += data['demands'][node_index]
            # plan_output += f' {node_index} -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        # plan_output += f'{manager.IndexToNode(index)}\n'
        # plan_output += f'Distance of route: {route_distance}\n'
        # plan_output += f'Load of route: {route_load}\n'
        # print(plan_output)
        total_distance += route_distance
        # total_load += route_load
    return total_distance
    # print(f'Total load of all routes: {total_load}')



In [ ]:
# # Create the data model
# data = create_data_model(d, demand, cap)

# # Create the routing index manager
# manager = pywrapcp.RoutingIndexManager(
#     len(data['distance_matrix']),
#     data['num_vehicles'],
#     data['depot'])

# # Create Routing Model
# routing = pywrapcp.RoutingModel(manager)

# # Create and register transit callback
# def distance_callback(from_index, to_index):
#     from_node = manager.IndexToNode(from_index)
#     to_node = manager.IndexToNode(to_index)
#     return data['distance_matrix'][from_node][to_node]

# transit_callback_index = routing.RegisterTransitCallback(distance_callback)
# routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# # Add Capacity constraint
# def demand_callback(from_index):
#     from_node = manager.IndexToNode(from_index)
#     return data['demands'][from_node]

# demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
# routing.AddDimensionWithVehicleCapacity(
#     demand_callback_index,
#     0,  # null capacity slack
#     data['vehicle_capacities'],  # vehicle maximum capacities
#     True,  # start cumul to zero
#     'Capacity')

# # Set initial solution
# initial_solution = create_initial_solution(manager, routing, rl_solution)

# # Setting search parameters
# search_parameters = pywrapcp.DefaultRoutingSearchParameters()
# search_parameters.first_solution_strategy = (
#     routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# search_parameters.local_search_metaheuristic = (
#     routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
# search_parameters.time_limit.FromSeconds(30)
# search_parameters.solution_limit = 200

# # Solve the problem
# solution = routing.SolveFromAssignmentWithParameters(
#     initial_solution, search_parameters)

# if solution:
#     print(print_solution(data, manager, routing, solution))
#     # print(solution)
# else:
#     print('No solution found!')
#     # return None


In [ ]:
# instances[3:4]

In [ ]:
gap_rl_or=[]
time_rl_or=[]
ans_rl_or=[]
for instance in instances:
  print(instance)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  policy = model.policy.to(device).eval()  # trained policy
  problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))
  # start_rl=time.time()
  td_reset = env.reset(vrplib_to_td(problem).to(device))
  with torch.inference_mode():
      out = policy(td_reset.clone(), env, decode_type="sampling", num_samples=128, select_best=True)
      unnormalized_td = env.reset(vrplib_to_td(problem, normalize=False).to(device))
      cost = -env.get_reward(unnormalized_td, out["actions"]).int().item()
  rl_solution = out["actions"][0].tolist()
  filename = os.path.join(path_to_save, instance+'.vrp')  # Replace with your actual file name
  d,demand,cap = parse_vry_file(filename)
  filenam = os.path.join(path_to_save, instance+'.sol')
  n_truck=parse_vry(filenam)
  d = convert_to_int(d)
  # Create the data model

  data = create_data_model(d, demand, cap)
  start_rl_or=time.time()
  # Create the routing index manager
  manager = pywrapcp.RoutingIndexManager(
      len(data['distance_matrix']),
      data['num_vehicles'],
      data['depot'])

  # Create Routing Model
  routing = pywrapcp.RoutingModel(manager)

  # Create and register transit callback
  def distance_callback(from_index, to_index):
      from_node = manager.IndexToNode(from_index)
      to_node = manager.IndexToNode(to_index)
      return data['distance_matrix'][from_node][to_node]

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  # Add Capacity constraint
  def demand_callback(from_index):
      from_node = manager.IndexToNode(from_index)
      return data['demands'][from_node]

  demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
  routing.AddDimensionWithVehicleCapacity(
      demand_callback_index,
      0,  # null capacity slack
      data['vehicle_capacities'],  # vehicle maximum capacities
      True,  # start cumul to zero
      'Capacity')

  # Set initial solution
  initial_solution = create_initial_solution(manager, routing, rl_solution)

  # Setting search parameters
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (
      routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  # search_parameters.time_limit.FromSeconds(30)
  search_parameters.solution_limit = 200

  # Solve the problem
  solution = routing.SolveFromAssignmentWithParameters(
      initial_solution, search_parameters)
  end_rl_or=time.time()-start_rl_or
  time_rl_or.append(end_rl_or)
  if solution:
      ans=print_solution(data, manager, routing, solution)
  else:
      ans=1e10
  sol = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
  optimal_cost = sol['cost']
  ans_rl_or.append(ans)
  gap=(ans-optimal_cost)/optimal_cost
  gap_rl_or.append(gap)


Li_21
Li_22
Li_23
Li_24
Li_25
Li_26
Li_27
Li_28
Li_29
Li_30
Li_31
Li_32


In [ ]:
import pandas as pd
df = pd.DataFrame({
    'Instance': instances,
    # 'No: of nodes': no_nodes,
    # 'BKS': bks,
    'RL OR sol': ans_rl_or,
    '% Gap RL  OR': gap_rl_or,
    'Time RL': time_rl_or
})

In [ ]:
df.to_csv('Result_b.csv', index=False)

In [ ]:
from google.colab import files
files.download('Result_b.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>